# Following is an example of how to build a safe route for biking 
> Stockholm is beautiful, but, as in most other pan-European cities, there are a lot of dangerouse sites for biking.
Wouldn't it be great if we could plan our trip avoiding these sites and consequently save lives!?


In this example, we'd like to showcase how to use the [directions API][directions] and to avoid a number of
dangerous sites while routing.

The challenge here is to prepare the data appropriately. I have chosen to plot a few places in the city where I feel unsafe biking or riding e-scooters to illustrate an example. This was done using http://geojson.io In the Geojson tool, I have placed markers where accidents occur. We would be able to make polygon (geofenced zones), but I choosed to place a marker on the map, and then make a radius of 30 meters surrounding the 

In [ ]:
import folium
import pyproj
import requests
from openrouteservice import client
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

## Set up the fundamentals

1. Get the data of 'dangerous places' 
2. Populate a 'dangerous site' buffer polygon list
    - Create buffer polygons around construction sites with 30 m radius and low resolution
    
This is how this would look like



In [ ]:
url = 'https://gist.githubusercontent.com/semljola/96516031eaeca3cd86521f228faf5209/raw/c26f53854b5cecd468935de0a79323e5d496730a/map.geojson'

def create_buffer_polygon(point_in, resolution=10, radius=10):
    convert = pyproj.Transformer.from_crs("epsg:4326", 'epsg:32632')  # WGS84 to UTM32N
    convert_back = pyproj.Transformer.from_crs('epsg:32632', "epsg:4326")  # UTM32N to WGS84
    point_in_proj = convert.transform(*point_in)
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)  # 10 m buffer

    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(convert_back.transform(*point))  # Transform back to WGS84

    return poly_wgs

In [ ]:
# Set up the fundamentals
api_key = 'api-key'  # Individual api key
ors = client.Client(key=api_key)  # Create client with api key
stockholm_json = requests.get(url).json()  # Get data as JSON

map_params = {'tiles': 'Stockholm',
              'location': ([59.332760, 18.076398]),
              'zoom_start': 15}
map1 = folium.Map(**map_params)

# Populate a dangerous site buffer polygon list
sites_poly = []
for site_data in stockholm_json['features']:
    site_coords = site_data['geometry']['coordinates']
    folium.features.Marker(list(reversed(site_coords)),
                           popup='Construction point<br>{0}'.format(site_coords)).add_to(map1)

    # Create buffer polygons around construction sites with 30 m radius and low resolution
    site_poly_coords = create_buffer_polygon(site_coords,
                                             resolution=2,  # low resolution to keep polygons lean
                                             radius=30)
    sites_poly.append(site_poly_coords)

    site_poly_coords = [(y, x) for x, y in site_poly_coords]  # Reverse coords for folium/Leaflet
    folium.vector_layers.Polygon(locations=site_poly_coords,
                                 color='#ffd699',
                                 fill_color='#ffd699',
                                 fill_opacity=0.2,
                                 weight=3).add_to(map1)

map1

One sensible thing one could do, is to eliminate dangerous zones which are not in the immediate surrounding of the route of interest. Hence, we can request a route without dangerous sites, take a reasonable buffer,
filter construction sites within the buffer and try again.

Let's try this:
1. Create new map to start from scratch
2. Request normal route between appropriate locations without danreous sites
    - Profile for route is set to 'cycling-regular', with preference set to 'shortest' path
3. Plot which dangerous sites fall into the buffer Polygon

Would look like

In [ ]:
# GeoJSON style function
def style_function(color):
    return lambda feature: dict(color=color,
                                weight=3,
                                opacity=0.5)


# Create new map to start from scratch
map_params.update({'location': ([59.334591, 18.063240]),
                   'zoom_start': 15})
map2 = folium.Map(**map_params)

# Request normal route between appropriate locations without construction sites
request_params = {'coordinates': [[18.080222, 59.338345],
                                  [18.074444, 59.326854]],
                  'format_out': 'geojson',
                  'profile': 'cycling-regular',
                  'preference': 'shortest',
                  'instructions': 'false', }
route_normal = ors.directions(**request_params)
folium.features.GeoJson(data=route_normal,
                        name='Route without construction sites',
                        style_function=style_function('#FF0000'),
                        overlay=True).add_to(map2)

# Buffer route with 0.009 degrees (really, just too lazy to project again...)
route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.009)
folium.features.GeoJson(data=geometry.mapping(route_buffer),
                        name='Route Buffer',
                        style_function=style_function('#FFFF00'),
                        overlay=True).add_to(map2)

# Plot which dangerous sites fall into the buffer Polygon
sites_buffer_poly = []
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map2)
        sites_buffer_poly.append(poly)

map2

Finally, we can try to request a route using `avoid_polygons`, which conveniently takes a GeoJSON as input.

In [19]:
# Add the site polygons to the request parameters
request_params['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly))}
route_detour = ors.directions(**request_params)

folium.features.GeoJson(data=route_detour,
                        name='Route with dangerouse sites',
                        style_function=style_function('#00FF00'),
                        overlay=True).add_to(map2)

map2.add_child(folium.map.LayerControl())
map2

> Note: This request might fail sometime in the future, as the JSON is loaded dynamically and changes a few times
> a week.
> Thus the amount of sites within the buffer can exceed the GET limit (which is between 15-20 site polygons approx).